In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionImg2ImgPipeline, AutoencoderKL,StableDiffusionPipeline

from transformers import AutoTokenizer, DistilBertForSequenceClassification
from diffusers import EulerDiscreteScheduler
import torch

In [ ]:
import torch
from PIL import Image
import numpy as np

In [ ]:
from upscale_utils import Upscale

In [ ]:
generator = DiffusionPipeline.from_pretrained("../model/diffuser_model/",torch_dtype=torch.float16,safety_checker=None)

# vae = AutoencoderKL.from_pretrained("vae_path",torch_dtype=torch.float16,subfolder="vae").to("cuda")
# generator.vae = vae

generator.scheduler = EulerDiscreteScheduler.from_config(generator.scheduler.config)

generator.to("cuda")

In [ ]:
upscale_model_dir_path = '../model/upscale'
upscale_generator = Upscale(upscale_model_dir_path)
#加载upscale模型
upscale_generator.upload_model_from_dir()
print(f'load upscale model:{len(upscale_generator.models)}')

In [ ]:
prompt = "(absurdres, highres, ultra detailed), 1 male, handsome, tall muscular guy, very short hair, best ratio four finger and one thumb, best light and shadow, background is back alley, detasiled sunlight, sitting, Little cats are gathered next to him, dappled sunlight, day, depth of field, plants, summer, (dutch angle), closed mouth, summer day"


negative = "(hair between eyes), sketch, duplicate, ugly, huge eyes, text, logo, worst face, (bad and mutated hands:1.3), (worst quality:2.0), (low quality:2.0), (blurry:2.0), horror, geometry, bad_prompt, (bad hands), (missing fingers), multiple limbs, bad anatomy, (interlocked fingers:1.2), Ugly Fingers, (extra digit and hands and fingers and legs and arms:1.4), ((2girl)), (deformed fingers:1.2), (long fingers:1.2), extra legs, upper teeth, parted lips, open mouth"


In [ ]:
import random

seed = int(random.randrange(4294967294))
seed=4196966724
print(seed)
Generator = [torch.Generator(device="cuda").manual_seed(i) for i in range(seed, seed + 1)]

#t2i
image = generator(prompt=prompt,
                  width=512,
                  height=768,
                  negative_prompt=negative,
                  num_inference_steps=20,
                  guidance_scale=9,
                  generator=Generator,
                  num_images_per_prompt=1).images

In [ ]:
imgs = [image[0],]
res_img = upscale_generator.upscale(imgs,scale_factor=2)
img = Image.fromarray(np.uint8(res_img[0]))

In [ ]:
stable_diffusion_img2img = StableDiffusionImg2ImgPipeline(
        vae=generator.vae,
        text_encoder=generator.text_encoder,
        tokenizer=generator.tokenizer,
        unet=generator.unet,
        scheduler=generator.scheduler,
        safety_checker=None,
        feature_extractor=None,
        requires_safety_checker=False,)

In [ ]:
images = stable_diffusion_img2img(
        prompt=prompt,
        image=img,
        strength=0.8,
        num_inference_steps=20,
        guidance_scale=9,
        negative_prompt=negative,
        num_images_per_prompt=1,
        generator=Generator
    ).images

In [ ]:
images[0]